In [2]:
import cv2
import mediapipe as mp
import serial
import time
from IPython.display import display, clear_output
from PIL import Image
import numpy as np


2025-10-22 12:53:46.014018: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-22 12:53:46.018565: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-22 12:53:46.034608: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761126826.060122   30864 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761126826.067112   30864 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761126826.087916   30864 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [3]:
arduino=serial.Serial('/dev/ttyACM1', 9600 , timeout=1)
time.sleep(2)

SerialException: [Errno 2] could not open port /dev/ttyACM1: [Errno 2] No such file or directory: '/dev/ttyACM1'

In [5]:

# --------- Mediapipe Setup ---------
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)

# Camera
cap = cv2.VideoCapture(0)

light_on = False

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = cv2.flip(frame, 1)  # mirror view
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb)
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Draw skeleton
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # Extract key landmarks: index finger tip (8) and thumb tip (4)
            index_tip = hand_landmarks.landmark[8]
            thumb_tip = hand_landmarks.landmark[4]
            
            # Check if "pinch" gesture (finger & thumb close together)
            if abs(index_tip.x - thumb_tip.x) < 0.05 and abs(index_tip.y - thumb_tip.y) < 0.05:
                light_on = not light_on
                if light_on:
                    print("ON")
                    arduino.write(b"ON\n")
                else:
                    print("OFF")
                    arduino.write(b"OFF\n")
                time.sleep(0.5)  # prevent flickering
    
    # Show status circle
    color = (0,255,0) if light_on else (0,0,255)
    cv2.circle(frame, (50,50), 30, color, -1)
    
    cv2.imshow("MediaPipe Hand Light Control", frame)
    
    if cv2.waitKey(1) & 0xFF == 27:  # ESC key
        break

cap.release()
cv2.destroyAllWindows()
arduino.close()


I0000 00:00:1761126858.181758   30864 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1761126858.185183   31080 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.0.7-0ubuntu0.24.04.2), renderer: Mesa Intel(R) HD Graphics 520 (SKL GT2)
[ WARN:0@33.222] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@33.223] global obsensor_uvc_stream_channel.cpp:159 getStreamChannelGroup Camera index out of range
W0000 00:00:1761126858.246062   31074 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1761126858.297861   31074 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


NameError: name 'arduino' is not defined